# Semana 1: Exploración del problema y del dataset

## 1. Cargar y visualizar señales ECG

In [ ]:
#Aqui va a estar el código

### Recomendación: usar matplotlib, wfdb, scipy.io

## 2. Resumen del artículo base (Clifford et al., 2017)

## 3. Análisis del dataset (cantidad de señales, duración, etiquetas, etc.)